### Notes 

T5 Paper: https://arxiv.org/pdf/1910.10683.pdf

T5 Tokenizer: https://github.com/huggingface/transformers/blob/master/src/transformers/tokenization_t5.py

Important Tasks: https://docs.google.com/document/d/1weIZM6QTlnitpPQmpg-WeV2RW70TnYmDuogBQPr5mB0/edit

In [51]:
#James Chartouni
#Joey Park
#Raef Khan

import torch
import pandas as pd
import numpy as np
import pickle
import os, io, glob

import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchtext.datasets.translation import TranslationDataset
from torchtext.data import Field, BucketIterator

import sentencepiece as spm

from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config


In [4]:
ls data/AD_NMT-master

LAV-MSA-2-both.pkl        Magribi_MSA-test.pkl      english-Arabic-train.pkl
LAV-MSA-2-test.pkl        Magribi_MSA-train.pkl     english-arabic-test.pkl
LAV-MSA-2-train.pkl       README.md
Magribi_MSA-both.pkl      english-Arabic-both.pkl


In [5]:
file_path = 'data/AD_NMT-master/'

with open(file_path + "english-Arabic-train.pkl", 'rb') as handle:
    train_ds = pickle.load(handle) 
    
with open(file_path + "english-Arabic-test.pkl", 'rb') as handle:
    test_ds = pickle.load(handle) 
    
with open(file_path + "LAV-MSA-2-both.pkl", 'rb') as handle:
    data_LAV_MSA = pickle.load(handle) 

with open(file_path + "english-Arabic-both.pkl", 'rb') as handle:
    data_English_MSA = pickle.load(handle) 
    
with open(file_path + "Magribi_MSA-both.pkl", 'rb') as handle:
    data_Magribi_MSA = pickle.load(handle) 
    

In [6]:
print(train_ds[0:5])
print(test_ds[0:5])

[['Tom was also there', 'كان توم هنا ايضا'], ['That old woman lives by herself', 'تلك المراة العجوز تسكن بمفردها'], ['He went abroad for the purpose of studying English', 'سافر خارج البلد ليتعلم الانجليزية'], ['There is a fork missing', 'هناك شوكة ناقصة'], ["I don't know this game", 'لا اعرف هذه اللعبة']]
[["Where's your money?", 'اين مالك؟'], ['Be prepared', 'كن مستعدا'], ["I figured you'd be impressed", 'توقعت انك ستنبهر'], ['May I come in?', 'هل بامكاني الدخول؟'], ['Read through the article', 'اقرا المقالة']]


## Prepare Datasets

example: https://iwslt2010.fbk.eu/node/32/

We need to take our training and test sets from the pkl files and create new .txt files that are formatted so that the standard torchtext Dataset class can read them

Data format:
each line consists of three fields divided by the character '\'
sentences consisting of words divided by single spaces
format: <SENTENCE_ID>\<PARAPHRASE_ID>\<TEXT>
Field_1: sentence ID
Field_2: paraphrase ID
Field_3: MT develop sentence / reference translation
Text input example:
DEV_001\01\This is the first develop sentence.
DEV_002\01\This is the second develop sentence.
Reference translation example:
DEV_001\01\1st reference translation for 1st input
DEV_001\02\2nd reference translation for 1st input
...
DEV_002\01\1st reference translation for 2nd input
DEV_002\02\2nd reference translation for 2nd input
...
Languages:
Arabic-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
IWSLT07 testset: 489 sentences, 6 reference translations
IWSLT08 testset: 507 sentences, 16 reference translations
French-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
Turkish-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations


In [7]:
file_path = 'data/'

In [8]:
def pytorch_format(ds,src='en',trg='ar',datatype=''):
  src_formatted = src + '_formatted_' + datatype + '.txt'
  trg_formatted = trg + '_formatted_' + datatype + '.txt'
  with open(file_path + src_formatted, 'wt') as enpy, open(file_path + trg_formatted, 'wt') as arpy:
    for i, arr in enumerate(ds):
      enpy.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[0] + '\n')
      arpy.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[1] + '\n')

In [9]:
pytorch_format(train_ds, 'en', 'ar', 'train')
pytorch_format(test_ds, 'en', 'ar', 'test')
pytorch_format(data_LAV_MSA, 'lav', 'msa', '')
pytorch_format(data_English_MSA, 'en', 'msa', '')
pytorch_format(data_Magribi_MSA, 'mag', 'msa', '')

In [23]:
cat data/lav_formatted_.txt

_0000\01\لا انا بعرف وحدة راحت ع فرنسا و معا شنتا حطت فيها الفرش
_0001\01\روح بوشك و فتول عاليسار
_0002\01\لا لا لازم انه يكون عندك موضوع ما في اشي
_0003\01\اوعي تبعدي من هون بلاش تضيعي 
_0004\01\قصدي صراحة يما انا كمان كرهته من يوم ما عملتيه زي ما بتعمله خالتي كرهته و صرت ما باطيقه بالمرة
_0005\01\قلي موضوعك كويس و كل شي و بكرا بردلك خبر اذا بقبلك او لا
_0006\01\كنت اذا بدي اقهر اخوتي اشتري كرتونة بوظة و بعدين اعمل حالي طيبة و اوزعها اخر الليل لاني متاكدة ما راح يصبح عليها صبح
_0007\01\انا كمان لما بداوا يطيبوا في الدار مرضت فيه انا ما فهمتش ليش
_0008\01\الكلمات حلوه والكليب احلى والاثنين بشجعوا على الحجاب وانا استغربت كتير من رايك بالاغنيه
_0009\01\بس قصة انو شغل محترم يعني بيت وسيارة 
_010\01\المهم الي خايف من الحرب خايف مشان عندو شي خايف يخسرو 
_011\01\الجزائر
_012\01\وعندك تبعت القصبة شوي
_013\01\المشكلة هان
_014\01\اشتغلي منيح
_015\01\لا لا اعذريني مين انت من وين اجيت و ايش بتعرف و ايش لازم انك تعمله ايش مالك انت بتحسب انه الشغل سهل في بلادنا بالمرة
_016\01\الله معك
_017\01\انا ه

_1877\01\وتوزيع الطلبة على اماكن واوقات معينة
_1878\01\و شو بالنسبة للمحاسبة
_1879\01\في حكاية بتزعل واحد اخد البيت من امو و تزوج
_1880\01\قصدي احترم حالك هاد كل شي
_1881\01\الزحمة و بتحسي انه الجزائر ما بتزيد عن 2 كم مربع
_1882\01\ما عمليش اشي بس هو مزعج
_1883\01\اذا هو قريب
_1884\01\بتقلك رح تجبلك ياه معها
_1885\01\الموفية اللي جبتلها اياها فتيحة من مكة
_1886\01\اي رح نروح بعدين
_1887\01\ما بقبل هون ترتيبي مو كويس
_1888\01\خاصة اذا بلشت اتعرق مو هيك ماما
_1889\01\اي هون الصعوبة
_1890\01\لما نخلص الدردشة لازم تقري النص و الشغلات اللي بتطلعلك بتستحق الدراسة ابعتيلي رسالة عشان ادرسها كمان
_1891\01\بالتاكيد قصة
_1892\01\اي حتى ياكلك واحد منهن
_1893\01\كريمة يا ريت لو طولتي رسالتك اللي بعتيها لكمال شي جملة وحدة بس
_1894\01\ماشي
_1895\01\ويا رب يكون في ميزان حسناتك 
_1896\01\مسكر مسكر مافيش ضواي
_1897\01\لكن قالولك شهر و بعدين
_1898\01\في الاخير
_1899\01\والمشاكل كبيرة كتير عشان هيك لازم نتلزم الاعتدال والتوسط
_1900\01\لازم تقنع اي حدا مختلف معك انو غلط وانت الصح
_1901\01\انا اخر واحد انصا

_2785\01\نشكر الله سهام انا اتصلت بنرجس هلا
_2786\01\خذ هذا حيحفظك
_2787\01\خلينا نقول حوالي اسبوع او اكتر
_2788\01\زعجوني شوي
_2789\01\كيف حيعرفوا انه جدي عايش
_2790\01\كمال  كان بدي اذكرك بالدعوة  بدي ابدا في الاجراءات  بتعرف انه كل اشي بدو وقت هان
_2791\01\شو الشي التاني لي كنت بدي قوله خلصتم من القاعة و الفرقة و هي الشغلات
_2792\01\اشرب اشرب التفاح طبيعي كتير كتير
_2793\01\حكت معي حوالي التنتين او الوحدة
_2794\01\ما فكرت اني لح انهان
_2795\01\اي
_2796\01\ليش ما تكملي هان ايش هو الفرق بين هان و هناك و هو انت جاهزة عشان تكملي هناك
_2797\01\ما شكلو هالشكل
_2798\01\قالوا لازم نقتلهم صاروا خطيرين
_2799\01\في ناس بيئولوا مات و ناس بيئولوا انو جن
_2800\01\الكل صار يضرب امتى ما يحلاله
_2801\01\بتعرفي باليوم الحدعش رح روح عفرنسا
_2802\01\اي نحن خلصنا تقريبا
_2803\01\يالله جاي
_2804\01\ايش انت تخينة انت عادية بس
_2805\01\مش متاكدة من انا بنقدر نكتبهم في السكايب
_2806\01\هو مزبوط يا اسامة انه سلو كانوا بيعيشوا في النرويج
_2807\01\ليش يا جماعة و ليش لم درسونا احنا الخوارزميات
_2808\01\سالوا ال

_7791\01\على فكرة هما كبروا فيرانها
_7792\01\يما ايش حنحضر حلويات
_7793\01\شكرا سارة شكرا كتير
_7794\01\لو تحكي معها الحقيقة يا وليك كانو بقرة نطحتك 
_7795\01\كل اشي فردي عنا
_7796\01\نظفتلك المعدة
_7797\01\انا لما روح عالدكتور شو قلو
_7798\01\الاسد ما بيزار الا ببابك يا امي
_7799\01\انت ارتبت
_7800\01\وعالعموم يا ابو حميد وانا بفتخر فيك كصديق واخ 
_7801\01\بظن بياكل العسل بادنيه
_7802\01\وغيرت الايميل اكتر من مرة 
_7803\01\صدقيني ما بعرف من زمان ما حاكيتا
_7804\01\ما حيتلي هالشي من ئبل
_7805\01\حلوه  مش هيك 
_7806\01\انشالله الله
_7807\01\بلحقكم الصبح و عالتلاتة و نص بتوقيت فرنسا
_7808\01\بظن انو الامريكان بدهون يضربوا بشار يا ريت يساووها
_7809\01\بالاخير ما تركوه و قالوله بتتزوج مرت اخوك اجباري
_7810\01\البسة احسن منك
_7811\01\عن شو بدنا نحكي اليوم
_7812\01\لو ماتت كان ممكن تموت شهيدة يعني ماتت شهيدة
_7813\01\فهمت كيف
_7814\01\ونسال الشخص الي زعلان كتير من الحكي 
_7815\01\محتارون الرياضة او الطب 
_7816\01\مزبوط مزبوط بتمر عليك اوقات بدك انك تفضفضي مع حدا
_7817\01\البنت اللي ما بتعجبه

_11155\01\قالتلي ب��نغاظ لحالي
_11156\01\لا صدقيني ولا اشي
_11157\01\انا هلقيت بديت في مجال وبديهي اني اواصل فيه
_11158\01\انتو اذا حبيت تختار شبكة لاسلكية بيعطوكو معدل جديد شي
_11159\01\اطلعي شوفي هيو في اختبار في قسنطينة و في اختبار في الجزائر كمان
_11160\01\رح نعليها لما توصل سطح كل مصري الو بيت 
_11161\01\اذا اجوكم بمسابقة خبريني
_11162\01\بعدا ما عاد حسيت بشي
_11163\01\لا انا عم بسالك شفتي مثلا عقدي عند الخاص
_11164\01\شو جابا لهون
_11165\01\شي يوم رح اهرب صدقني ما رح تلاقيني
_11166\01\الظاهر انو مش مهتمة فيها
_11167\01\بارك الله فيك قلتلها يا سيدتي كل اشي عولج في الاعلام الالي بس بيضل دائما هناك امور اضافية لما اخلص شغلي حاخصص فصل فيه مقارنة بين شغلي و شغل اللي سبقوني و نشوف مين الاحسن اذا هذه كمان سيبنا منها
_11168\01\قال حسيتب بوجع
_11169\01\ههه بارد علاخر يا منى بسك
_11170\01\يكونوا متاكدين من اخلاقه ومن عيلته
_11171\01\ليش الكتو
_11172\01\بعدين اخدوها عالمشفى
_11173\01\راحت الريحة بطلت موجودة
_11174\01\اوعك يفكر انك مجنونة بيقول ما بيجينيش الا المجانين
_11175\01\اكيد بس الاهم

## Build Vocabulary 

Sentence Piece Google Colab
https://github.com/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb

In [10]:
len(data_English_MSA)

10001

In [11]:
"""
Create a text file with all the MSA vocab available for SentencePiece to create a library 
"""

text_file_en = open("data/english_data.txt", "wt")
text_file_msa = open("data/arabic_data.txt", "wt")

MSA_text = ""
EN_text = ""

for line in data_English_MSA:
        english = line[0]
        english_words = english.split(" ")
        for count, word in enumerate(english_words):
            text_file_en.write(word)
        text_file_en.write("\n")
        
        arabic = line[1]
        arabic_words = arabic.split(" ")
        for count, word in enumerate(arabic_words):
            text_file_msa.write(word)
        text_file_msa.write("\n")


text_file_en.close()
text_file_msa.close()

In [12]:
spm.SentencePieceTrainer.train('--input=data/arabic_data.txt --model_prefix=data/msa --vocab_size=2000')
spm.SentencePieceTrainer.train('--input=data/english_data.txt --model_prefix=data/en --vocab_size=2000')

True

In [13]:
sp = spm.SentencePieceProcessor()
sp.load('data/en.model')

True

In [14]:
ls

Env_Setup_Instructions.txt  data/
Model_1.ipynb               transformers/
README.md


In [15]:
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

['▁This', '▁', 'is', '▁', 'a', '▁', 't', 'est']
[89, 83, 12, 83, 8, 83, 6, 309]


## TF Tokenizer

https://huggingface.co/transformers/model_doc/t5.html#t5tokenizer

In [16]:
ls data/

AD_NMT-master/
AD_NMT-master.zip
PADIC.xml
UNv1.0.ar-en.tar.gz.00
UNv1.0.ar-en.tar.gz.01
ar_formatted_test.txt
ar_formatted_train.txt
arabic-to-english-translation-sentences.zip
arabic_data.txt
en.model
en.vocab
en_formatted_.txt
en_formatted_test.txt
en_formatted_train.txt
english_data.txt
lav_formatted_.txt
mag_formatted_.txt
msa.model
msa.vocab
msa_formatted_.txt


In [17]:
msa_tokenizer = T5Tokenizer("data/msa.model")
en_tokenizer = T5Tokenizer("data/en.model")

In [18]:
input_ids = msa_tokenizer.encode('هل بامكاني الدخول؟ </s>', return_tensors='pt')

In [19]:
input_ids

tensor([[ 41,   4, 330,   6,  93, 964,   8,   4,   2]])

In [ ]:
#decode to make sure you can go back and forth between the encoding properly 
#@raef can you verify the Arabic?

In [25]:
ls data/

AD_NMT-master/
AD_NMT-master.zip
PADIC.xml
UNv1.0.ar-en.tar.gz.00
UNv1.0.ar-en.tar.gz.01
ar_formatted_test.txt
ar_formatted_train.txt
arabic-to-english-translation-sentences.zip
arabic_data.txt
en.model
en.vocab
en_formatted_.txt
en_formatted_test.txt
en_formatted_train.txt
english_data.txt
lav_formatted_.txt
mag_formatted_.txt
msa.model
msa.vocab
msa_formatted_.txt


## Pytorch Data Set and Data Loader

https://github.com/google-research/text-to-text-transfer-transformer

pytorch dataset: https://pytorch.org/text/_modules/torchtext/datasets/translation.html

pytorch dataset documentation: https://torchtext.readthedocs.io/en/latest/datasets.html#iwslt

example dataset: https://iwslt2010.fbk.eu/node/32/

Field API: https://pytorch.org/text/data.html#torchtext.data.Field
Field Source: https://pytorch.org/text/_modules/torchtext/data/field.html#Field

https://pytorch.org/text/_modules/torchtext/data/utils.html#get_tokenizer
    
we need to pass our tokenizer as a function     

In [57]:

SRC = Field(tokenize = T5Tokenizer,
            tokenizer_language="en",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

TRG = Field(tokenize = T5Tokenizer,
            tokenizer_language="ar",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

In [58]:
dataset = TranslationDataset(path='data/', exts=('en_formatted_.txt', 'msa_formatted_.txt'), fields=(SRC, TRG))

OSError: Not found: "_0000\01\Tom was also there": No such file or directory Error #2

In [ ]:
#For testing dataset code errors. Delete when dataset is functioning properly 

path = 'data/'
exts=('en_formatted_.txt', 'msa_formatted_.txt')
fields=(' sentence ID', 'paraphrase ID', 'MT training sentence')

if not isinstance(fields[0], (tuple, list)):
    fields = [('src', fields[0]), ('trg', fields[1])]

src_path, trg_path = tuple(os.path.expanduser(path + x) for x in exts)

examples = []
with io.open(src_path, mode='r', encoding='utf-8') as src_file, \
        io.open(trg_path, mode='r', encoding='utf-8') as trg_file:
    for src_line, trg_line in zip(src_file, trg_file):
        src_line, trg_line = src_line.strip(), trg_line.strip()
        if src_line != '' and trg_line != '':
            print(src_line)
            print(trg_line)
            examples.append(data.Example.fromlist(
                [src_line, trg_line], fields))

In [55]:
train_data, valid_data, test_data = dataset.splits(exts=('en_formatted_.txt', 'msa_formatted_.txt'),
                                                    fields = (SRC, TRG))

NameError: name 'dataset' is not defined

In [0]:
BATCH_SIZE =32

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

## Model Training

https://huggingface.co/transformers/model_doc/t5.html#training

In [0]:
configuration = T5Config()

In [0]:
model = T5Model.from_pretrained('t5-base')

In [0]:
for i, batch in enumerate(train_iterator)

## Model Inference

https://github.com/huggingface/transformers/blob/master/examples/translation/t5/evaluate_wmt.py